##### Hello, MNIST

In [27]:
# import mnist dataset from sklearn
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
x, y = mnist['data'], mnist['target']
x.shape, y.shape

((70000, 784), (70000,))

In [28]:
print(type(x))
print(type(y))
print(y.dtypes)
print([type(i) for i in y[:1]])

# since y' dtype is str, we need to convert it to float
y = y.astype(float)
print([type(i) for i in y[:1]])

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
category
[<class 'str'>]
[<class 'float'>]


In [29]:
# normalize x to keep our gradients manageable
x = x / 255

In [31]:
# to build a zero-classifier, make the label 1 when we have a zero, and 0 otherwise
import numpy as np
y_new = np.zeros(shape=y.shape)
y_new[np.where(y == 0.0)] = 1
y = y_new